## Corpora

In [1]:
from ipynb.fs.full.koselleck import *
# pd.options.display.latex.repr=True
ybin=5

[Koselleck] (21:22:03) Alles bereit 


In [2]:
C=None
def get_corpus():
    global C
    if C is None:
        C=lltk.load(CNAME)
    return C

In [3]:
C=get_corpus()

## OCR

In [4]:
from lltk import get_wordlist

In [5]:
def measure_ocr_accuracy(txt_or_tokens_or_tokend,lang='en'):
    wordlist={w.lower() for w in get_wordlist(lang=lang)}
    if type(txt_or_tokens_or_tokend) in {str}:
        tokend=Counter(tokenize(txt_or_tokens_or_tokend))
    elif type(txt_or_tokens_or_tokend) in {tuple,list}:
        tokens=Counter(txt_or_tokens_or_tokend)
    elif type(txt_or_tokens_or_tokend) in {dict,Counter}:
        tokend=txt_or_tokens_or_tokend
    else: 
        raise Exception("Function `measure_ocr_accuracy(txt_or_tokens_or_tokend)` must take text string or list of tokens or dictionary.")
    numwords=sum(tokend.values())
    numrecog=sum(
        val for tok,val in tokend.items()
        if tok.lower() in wordlist
    )
    return {
        'num_words':numwords,
        'num_recog':numrecog,
        'ocr_accuracy':numrecog/numwords
    }

In [6]:
def writeiter_jsonl(fnfn,iterr):
    import jsonlines
    with jsonlines.open(fnfn,'w') as writer:
        for i,dx in enumerate(iterr):
            writer.write(dx)

In [7]:

def do_measure_ocr_accuracy_corpus(obj):
    idx,path_freqs=obj
    od={'id':idx}
    if not os.path.exists(path_freqs): return od
    with open(path_freqs) as f: d=ujson.load(f)
    odx=measure_ocr_accuracy(d)
    return {**od, **odx}
    
def measure_ocr_accuracy_corpus(self,num_proc=1,**y):
    ofn=os.path.join(self.path_data, 'data.ocr_accuracy.jsonl')
    if os.path.exists(ofn): return pd.read_json(ofn,lines=True).set_index('id')
    objs=[(t.id,t.path_freqs) for t in self.texts()]
    iterr=pmap_iter(
        do_measure_ocr_accuracy_corpus,
        objs,
        num_proc=num_proc,
        **y
    )
    return writeiter_jsonl(ofn, iterr)

In [8]:
# dfocr=measure_ocr_accuracy_corpus(C,num_proc=4)
# dfocr

In [9]:
# stop

In [10]:
# !cat /home/ryan/lltk_data/corpora/bpo/data/data.ocr_accuracy.jsonl

In [11]:
# measure_ocr_accuracy(d)

In [12]:
# stop

In [13]:
dfmeta_orig=read_df('/home/ryan/lltk_data/corpora/bpo/metadata-full.csv')
dfmeta_orig

,NumericPubDate,RecordID,RecordTitle,Publication_PublicationID,Publication_Title,Publication_Qualifier,Publisher,VolumeIssue,StartPage,EndPage,ObjectType,ISSN,LanguageCode,SourceType,num_words
0,19061020,7696950,Advertisement,2563,Musical standard,London,Reeves and Turner,NaN,[iii],[iii],Advertisement,2043-4847,ENG,Historical Periodicals,1209
1,18971201,8099455,FINGERING THE VIOLA.,2896,The Violin times : a monthly journal for professional and amateur violinists and quartet players,London,W. Reeves,NaN,32,32,General Information,2044-6608,ENG,Historical Periodicals,494
2,18800103,9464136,BEKKELEY CASTLE.,13715,"Saturday review of politics, literature, science and art",London,J.W. Parker and Son,NaN,12,14,NaN,0950-9852,ENG,Historical Periodicals,3762
3,19500322,1618790372,"Macmillan & Co., Ltd.",2032697,The Tatler and Bystander,NaN,NaN,NaN,562,NaN,Advertisement,NaN,ENG,Historical Periodicals,95
4,18990601,7783559,Sweet Vales of Devonia.,2567,"Musical times and singing class circular, 1844-1903",London / New York,Novello,NaN,1,4,Poem,NaN,ENG,Historical Periodicals,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2869239,19020322,9324438,Advertisement,13715,"Saturday review of politics, literature, science and art",London,J.W. Parker and Son,NaN,383,383,Advertisement,0950-9852,ENG,Historical Periodicals,2381
2869240,19580924,1618649620,PETER SCOTT & CO LTD,2032697,The Tatler and Bystander,NaN,NaN,NaN,619,NaN,Advertisement,NaN,ENG,Historical Periodicals,33
2869241,19180921,1618735471,MELLIN'S FOOD WORKS,2032689,The Sphere,London,Illustrated London News,NaN,i,NaN,Advertisement,NaN,ENG,Historical Periodicals,65
2869242,18970929,1638088199,SMALL TALK,2032690,The Sketch,London,Illustrated London News,NaN,389,NaN,NaN,NaN,ENG,Historical Periodicals,6776


In [14]:
dfmeta_orig['year']=dfmeta_orig['NumericPubDate'].apply(lambda x: int(str(x)[:4]))
dfmeta_orig['period']=dfmeta_orig.year.apply(lambda y: periodize_bystep(y,25,1600))
dfmeta_orig

,NumericPubDate,RecordID,RecordTitle,Publication_PublicationID,Publication_Title,Publication_Qualifier,Publisher,VolumeIssue,StartPage,EndPage,ObjectType,ISSN,LanguageCode,SourceType,num_words,year,period
0,19061020,7696950,Advertisement,2563,Musical standard,London,Reeves and Turner,NaN,[iii],[iii],Advertisement,2043-4847,ENG,Historical Periodicals,1209,1906,1900-1925
1,18971201,8099455,FINGERING THE VIOLA.,2896,The Violin times : a monthly journal for professional and amateur violinists and quartet players,London,W. Reeves,NaN,32,32,General Information,2044-6608,ENG,Historical Periodicals,494,1897,1875-1900
2,18800103,9464136,BEKKELEY CASTLE.,13715,"Saturday review of politics, literature, science and art",London,J.W. Parker and Son,NaN,12,14,NaN,0950-9852,ENG,Historical Periodicals,3762,1880,1875-1900
3,19500322,1618790372,"Macmillan & Co., Ltd.",2032697,The Tatler and Bystander,NaN,NaN,NaN,562,NaN,Advertisement,NaN,ENG,Historical Periodicals,95,1950,1950-1975
4,18990601,7783559,Sweet Vales of Devonia.,2567,"Musical times and singing class circular, 1844-1903",London / New York,Novello,NaN,1,4,Poem,NaN,ENG,Historical Periodicals,35,1899,1875-1900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2869239,19020322,9324438,Advertisement,13715,"Saturday review of politics, literature, science and art",London,J.W. Parker and Son,NaN,383,383,Advertisement,0950-9852,ENG,Historical Periodicals,2381,1902,1900-1925
2869240,19580924,1618649620,PETER SCOTT & CO LTD,2032697,The Tatler and Bystander,NaN,NaN,NaN,619,NaN,Advertisement,NaN,ENG,Historical Periodicals,33,1958,1950-1975
2869241,19180921,1618735471,MELLIN'S FOOD WORKS,2032689,The Sphere,London,Illustrated London News,NaN,i,NaN,Advertisement,NaN,ENG,Historical Periodicals,65,1918,1900-1925
2869242,18970929,1638088199,SMALL TALK,2032690,The Sketch,London,Illustrated London News,NaN,389,NaN,NaN,NaN,ENG,Historical Periodicals,6776,1897,1875-1900


In [15]:
# orig_stats = pd.DataFrame(
#     dict(
#         period=i,
#         num_journals=gdf.Publication_PublicationID.nunique(),
#         num_articles=gdf.RecordID.nunique()
#     ) for i,gdf in dfmeta_orig.groupby('period')
# ).set_index('period')
# orig_stats

In [16]:
idf=dfmeta_orig.query('1670<=year<2000')
idf['num_journals']=idf['Publication_PublicationID']
idf['num_articles']=idf['RecordID']
idf['period']=idf.year.apply(lambda y: periodize(y,ybin))
orig_stats=idf[['period','num_journals','num_articles']].groupby('period').nunique()#.iloc[1:]
orig_stats

,num_journals,num_articles
period,,
1680-1685,4,1672
1685-1690,1,603
1690-1695,11,1211
1695-1700,8,432
1700-1705,9,1692
...,...,...
1970-1975,1,582
1980-1985,1,48
1985-1990,1,65


In [17]:
# C.meta.query('1710<year<1720').Publication_Title.value_counts()
meta=C.meta.query('1600<=year<2100')

In [18]:
s=meta.Publication_Title.value_counts()
# s

In [43]:
dfpub2=meta.groupby('Publication_Title').agg({'year':['min','median','max']})
dfpub2.columns=['_'.join(x) for x in dfpub2.columns.to_flat_index()]
dfpub2['size']=s
dfpub2=dfpub2.sort_values('size',ascending=False)
dfpub2

,year_min,year_median,year_max,size
Publication_Title,,,,
"The Gentleman's Magazine: and historical chronicle, Jan. 1736-Dec. 1833",1736,1784.0,1833,811
The Athenaeum,1830,1876.0,1921,724
Illustrated Sporting and Dramatic News,1642,1905.0,1943,672
Examiner,1808,1845.0,1881,595
The Sketch,1864,1925.0,1997,584
...,...,...,...,...
"General history of discoveries and improvements : in useful arts, particularly in the great branches of commerce, navigation, and plantation, in all parts of the known world",1726,1726.0,1726,1
Fortnightly,1934,1934.0,1934,1
"The History of learning, 1694",1694,1694.0,1694,1


In [44]:
dfpub2.query('year_min<1700').sort_values('year_min')

,year_min,year_median,year_max,size
Publication_Title,,,,
Illustrated Sporting and Dramatic News,1642,1905.0,1943,672
Heraclitus ridens,1681,1703.0,1718,15
"Democritus ridens, or, Comus and Momus",1681,1681.0,1681,3
"The Observator in question and answer, Apr. 1681- July 1681",1681,1681.0,1681,6
The Observator in dialogue,1682,1685.0,1687,20
"Weekly memorials for the ingenious : or, An account of books lately set forth in several languages",1682,1682.0,1683,10
"Weekly lampoon; or, Satyrical reflections on the last weeks publick news letters and observator, Oct. 1690",1690,1690.0,1690,3
"The History of learning, or, An abstract of several books lately published, as well abroad, as at home / By one of the two authors of the Universal and historical bibliotheque",1691,1691.0,1691,2
Athenian gazette,1691,1693.0,1697,36


## Corp info

In [45]:
# C.meta.path_freqs

In [46]:
C.meta[C.meta.Publication_Title.str.contains('Spectator')]

,author,title,year,NumericPubDate,RecordID,Publication_PublicationID,Publication_Title,Publication_Qualifier,Publisher,VolumeIssue,...,ISSN,LanguageCode,SourceType,num_words,path_freqs,path_txt,path_xml,_year_orig,corpus,Genre
id,,,,,,,,,,,,,,,,,,,,,
1711/Unknown/18038,,Untitled item,1711,17110303,6319128,18038,The Spectator (1711),London,Sam. Buckley.,,...,1358-6149,ENG,Historical Periodicals,1244,/home/ryan/lltk_data/corpora/bpo/freqs/1711/Unknown/18038.json,/home/ryan/lltk_data/corpora/bpo/txt/1711/Unknown/18038.txt,/home/ryan/lltk_data/corpora/bpo/xml/1711/Unknown/18038.xml,1711,BPO,Undefined
1711/Back_Matter/18038,,Back matter,1711,17110303,6319159,18038,The Spectator (1711),London,Sam. Buckley.,,...,1358-6149,ENG,Historical Periodicals,29,/home/ryan/lltk_data/corpora/bpo/freqs/1711/Back_Matter/18038.json,/home/ryan/lltk_data/corpora/bpo/txt/1711/Back_Matter/18038.txt,/home/ryan/lltk_data/corpora/bpo/xml/1711/Back_Matter/18038.xml,1711,BPO,Back Matter
1711/Review/18038,,Book review,1711,17110305,6313768,18038,The Spectator (1711),London,Sam. Buckley.,,...,1358-6149,ENG,Historical Periodicals,1704,/home/ryan/lltk_data/corpora/bpo/freqs/1711/Review/18038.json,/home/ryan/lltk_data/corpora/bpo/txt/1711/Review/18038.txt,/home/ryan/lltk_data/corpora/bpo/xml/1711/Review/18038.xml,1711,BPO,Review
1711/Advertisement/18038,,Advertisement,1711,17110306,6317135,18038,The Spectator (1711),London,Sam. Buckley.,,...,1358-6149,ENG,Historical Periodicals,833,/home/ryan/lltk_data/corpora/bpo/freqs/1711/Advertisement/18038.json,/home/ryan/lltk_data/corpora/bpo/txt/1711/Advertisement/18038.txt,/home/ryan/lltk_data/corpora/bpo/xml/1711/Advertisement/18038.xml,1711,BPO,Advertisement
1711/Correspondence/18038,,Letter,1711,17110309,6399178,18038,The Spectator (1711),London,Sam. Buckley.,,...,1358-6149,ENG,Historical Periodicals,1187,/home/ryan/lltk_data/corpora/bpo/freqs/1711/Correspondence/18038.json,/home/ryan/lltk_data/corpora/bpo/txt/1711/Correspondence/18038.txt,/home/ryan/lltk_data/corpora/bpo/xml/1711/Correspondence/18038.xml,1711,BPO,Correspondence
1711/General_Information/18038,,ERRATA.,1711,17110416,6314035,18038,The Spectator (1711),London,Sam. Buckley.,,...,1358-6149,ENG,Historical Periodicals,32,/home/ryan/lltk_data/corpora/bpo/freqs/1711/General_Information/18038.json,/home/ryan/lltk_data/corpora/bpo/txt/1711/General_Information/18038.txt,/home/ryan/lltk_data/corpora/bpo/xml/1711/General_Information/18038.xml,1711,BPO,General Information
1712/Correspondence/18038,,Letter,1712,17120101,6322677,18038,The Spectator (1711),London,Sam. Buckley.,,...,1358-6149,ENG,Historical Periodicals,1652,/home/ryan/lltk_data/corpora/bpo/freqs/1712/Correspondence/18038.json,/home/ryan/lltk_data/corpora/bpo/txt/1712/Correspondence/18038.txt,/home/ryan/lltk_data/corpora/bpo/xml/1712/Correspondence/18038.xml,1712,BPO,Correspondence
1712/Back_Matter/18038,,Back matter,1712,17120101,6413487,18038,The Spectator (1711),London,Sam. Buckley.,,...,1358-6149,ENG,Historical Periodicals,39,/home/ryan/lltk_data/corpora/bpo/freqs/1712/Back_Matter/18038.json,/home/ryan/lltk_data/corpora/bpo/txt/1712/Back_Matter/18038.txt,/home/ryan/lltk_data/corpora/bpo/xml/1712/Back_Matter/18038.xml,1712,BPO,Back Matter
1712/Advertisement/18038,,Advertisement,1712,17120102,6403376,18038,The Spectator (1711),London,Sam. Buckley.,,...,1358-6149,ENG,Historical Periodicals,319,/home/ryan/lltk_data/corpora/bpo/freqs/1712/Advertisement/18038.json,/home/ryan/lltk_data/corpora/bpo/txt/1712/Advertisement/18038.txt,/home/ryan/lltk_data/corpora/bpo/xml/1712/Advertisement/18038.xml,1712,BPO,Advertisement


In [47]:
# import ujson

# def gettextlen(path):
#     try:
#         with open(path) as f:
#             d=ujson.load(f)
#             return sum(d.values()) if d else np.nan
#     except FileNotFoundError:
#         return np.nan
    
# text_lens = pmap(gettextlen, C.meta.path_freqs)

In [48]:
dfmeta=C.meta
dfmeta['Genre']=dfmeta.ObjectType.replace({'':'Undefined'})
dfmeta

,author,title,year,NumericPubDate,RecordID,Publication_PublicationID,Publication_Title,Publication_Qualifier,Publisher,VolumeIssue,...,ISSN,LanguageCode,SourceType,num_words,path_freqs,path_txt,path_xml,_year_orig,corpus,Genre
id,,,,,,,,,,,,,,,,,,,,,
1010/Advertisement/2032695,,Advertisements,1010,10101,1618364638,2032695,Illustrated Sporting and Dramatic News,London,Illustrated London News,,...,,ENG,Historical Periodicals,1155,/home/ryan/lltk_data/corpora/bpo/freqs/1010/Advertisement/2032695.json,/home/ryan/lltk_data/corpora/bpo/txt/1010/Advertisement/2032695.txt,/home/ryan/lltk_data/corpora/bpo/xml/1010/Advertisement/2032695.xml,1010,BPO,Advertisement
1010/Advertisement/2032690,,MAPPIN & WEBB Ltd.,1010,10101,1618367119,2032690,The Sketch,London,Illustrated London News,,...,,ENG,Historical Periodicals,131,/home/ryan/lltk_data/corpora/bpo/freqs/1010/Advertisement/2032690.json,/home/ryan/lltk_data/corpora/bpo/txt/1010/Advertisement/2032690.txt,/home/ryan/lltk_data/corpora/bpo/xml/1010/Advertisement/2032690.xml,1010,BPO,Advertisement
1010/Unknown/2032695,,THE Old Etonians beat Edinburgh University by five goal four in a football match at Edinburgh on Tuesday,1010,10101,1618367761,2032695,Illustrated Sporting and Dramatic News,London,Illustrated London News,,...,,ENG,Historical Periodicals,55,/home/ryan/lltk_data/corpora/bpo/freqs/1010/Unknown/2032695.json,/home/ryan/lltk_data/corpora/bpo/txt/1010/Unknown/2032695.txt,/home/ryan/lltk_data/corpora/bpo/xml/1010/Unknown/2032695.xml,1010,BPO,Undefined
1010/News/2032690,,"GIVEN COMING-OF,-AGE FESTIVITIES, JUST LIKE A SON AND HEIR",1010,10101,1618367834,2032690,The Sketch,London,Illustrated London News,,...,,ENG,Historical Periodicals,64,/home/ryan/lltk_data/corpora/bpo/freqs/1010/News/2032690.json,/home/ryan/lltk_data/corpora/bpo/txt/1010/News/2032690.txt,/home/ryan/lltk_data/corpora/bpo/xml/1010/News/2032690.xml,1010,BPO,News
1010/News/2032695,,Graphic,1010,10101,1618372563,2032695,Illustrated Sporting and Dramatic News,London,Illustrated London News,,...,,ENG,Historical Periodicals,0,/home/ryan/lltk_data/corpora/bpo/freqs/1010/News/2032695.json,/home/ryan/lltk_data/corpora/bpo/txt/1010/News/2032695.txt,/home/ryan/lltk_data/corpora/bpo/xml/1010/News/2032695.xml,1010,BPO,News
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996/Advertisement/2032697,,TURKEY,1996,19960226,1689079340,2032697,London Life,London,,,...,,ENG,Historical Periodicals,233,/home/ryan/lltk_data/corpora/bpo/freqs/1996/Advertisement/2032697.json,/home/ryan/lltk_data/corpora/bpo/txt/1996/Advertisement/2032697.txt,/home/ryan/lltk_data/corpora/bpo/xml/1996/Advertisement/2032697.xml,1996,BPO,Advertisement
1996/Unknown/2032697,,BELGRAVIA DECLARES METER WAR,1996,19960226,1689080163,2032697,London Life,London,,,...,,ENG,Historical Periodicals,894,/home/ryan/lltk_data/corpora/bpo/freqs/1996/Unknown/2032697.json,/home/ryan/lltk_data/corpora/bpo/txt/1996/Unknown/2032697.txt,/home/ryan/lltk_data/corpora/bpo/xml/1996/Unknown/2032697.xml,1996,BPO,Undefined
1996/Review/2032697,,The Absorbing Fire,1996,19960226,1689080953,2032697,London Life,London,,,...,,ENG,Historical Periodicals,68,/home/ryan/lltk_data/corpora/bpo/freqs/1996/Review/2032697.json,/home/ryan/lltk_data/corpora/bpo/txt/1996/Review/2032697.txt,/home/ryan/lltk_data/corpora/bpo/xml/1996/Review/2032697.xml,1996,BPO,Review


In [49]:
ybin=20
dfmeta=dfmeta.query('1675<=year<1975')
dfmeta['period']=dfmeta.year.apply(lambda y: periodize(y,ybin))
dfmeta

,author,title,year,NumericPubDate,RecordID,Publication_PublicationID,Publication_Title,Publication_Qualifier,Publisher,VolumeIssue,...,LanguageCode,SourceType,num_words,path_freqs,path_txt,path_xml,_year_orig,corpus,Genre,period
id,,,,,,,,,,,,,,,,,,,,,
1681/Advertisement/18561,,Advertisement,1681,16810103,5975437,18561,Heraclitus ridens,London,,,...,ENG,Historical Periodicals,126,/home/ryan/lltk_data/corpora/bpo/freqs/1681/Advertisement/18561.json,/home/ryan/lltk_data/corpora/bpo/txt/1681/Advertisement/18561.txt,/home/ryan/lltk_data/corpora/bpo/xml/1681/Advertisement/18561.xml,1681,BPO,Advertisement,1680-1700
1681/Back_Matter/18561,,Back matter,1681,16810110,5975770,18561,Heraclitus ridens,London,,,...,ENG,Historical Periodicals,9,/home/ryan/lltk_data/corpora/bpo/freqs/1681/Back_Matter/18561.json,/home/ryan/lltk_data/corpora/bpo/txt/1681/Back_Matter/18561.txt,/home/ryan/lltk_data/corpora/bpo/xml/1681/Back_Matter/18561.xml,1681,BPO,Back Matter,1680-1700
1681/Unknown/18561,,Cover,1681,16810201,6042566,18561,Heraclitus ridens,London,,,...,ENG,Historical Periodicals,51,/home/ryan/lltk_data/corpora/bpo/freqs/1681/Unknown/18561.json,/home/ryan/lltk_data/corpora/bpo/txt/1681/Unknown/18561.txt,/home/ryan/lltk_data/corpora/bpo/xml/1681/Unknown/18561.xml,1681,BPO,Undefined,1680-1700
1681/Back_Matter/7766,,Back matter,1681,16810401,3912492,7766,"The Observator in question and answer, Apr. 1681- July 1681",London,C. Brome,,...,ENG,Historical Periodicals,10,/home/ryan/lltk_data/corpora/bpo/freqs/1681/Back_Matter/7766.json,/home/ryan/lltk_data/corpora/bpo/txt/1681/Back_Matter/7766.txt,/home/ryan/lltk_data/corpora/bpo/xml/1681/Back_Matter/7766.xml,1681,BPO,Back Matter,1680-1700
1681/General_Information/7766,,Untitled item,1681,16810401,3913242,7766,"The Observator in question and answer, Apr. 1681- July 1681",London,C. Brome,,...,ENG,Historical Periodicals,2041,/home/ryan/lltk_data/corpora/bpo/freqs/1681/General_Information/7766.json,/home/ryan/lltk_data/corpora/bpo/txt/1681/General_Information/7766.txt,/home/ryan/lltk_data/corpora/bpo/xml/1681/General_Information/7766.xml,1681,BPO,General Information,1680-1700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1974/Review/2032695,,Music,1974,19740718,1689169289,2032695,Farm & Country,London,Illustrated London News,,...,ENG,Historical Periodicals,1520,/home/ryan/lltk_data/corpora/bpo/freqs/1974/Review/2032695.json,/home/ryan/lltk_data/corpora/bpo/txt/1974/Review/2032695.txt,/home/ryan/lltk_data/corpora/bpo/xml/1974/Review/2032695.xml,1974,BPO,Review,1960-1980
1974/Unknown/2032695,,HER MAJESTY'S THEATRE,1974,19740718,1689169404,2032695,Farm & Country,London,Illustrated London News,,...,ENG,Historical Periodicals,84,/home/ryan/lltk_data/corpora/bpo/freqs/1974/Unknown/2032695.json,/home/ryan/lltk_data/corpora/bpo/txt/1974/Unknown/2032695.txt,/home/ryan/lltk_data/corpora/bpo/xml/1974/Unknown/2032695.xml,1974,BPO,Undefined,1960-1980
1974/Advertisement/2032695,,A NEW SADDLE PASTE,1974,19740718,1689181689,2032695,Farm & Country,London,Illustrated London News,,...,ENG,Historical Periodicals,78,/home/ryan/lltk_data/corpora/bpo/freqs/1974/Advertisement/2032695.json,/home/ryan/lltk_data/corpora/bpo/txt/1974/Advertisement/2032695.txt,/home/ryan/lltk_data/corpora/bpo/xml/1974/Advertisement/2032695.xml,1974,BPO,Advertisement,1960-1980


In [50]:
dfmeta['count']=1
dfinfo1=dfmeta.groupby('period').sum().drop('year',1)
dfinfo1

,count
period,
1680-1700,174
1700-1720,493
1720-1740,758
1740-1760,1143
1760-1780,1633
1780-1800,2280
1800-1820,3773
1820-1840,5102
1840-1860,5901


## Skip info

In [51]:
# get_skipgrams??

In [52]:
dfskip=get_skipgrams(calc_numlines=True)
dfskip=dfskip.query('1600<year<2000')
dfskip['period']=dfskip.year.apply(lambda y: periodize(y,ybin))

100%|██████████| 289/289 [00:21<00:00, 13.63it/s]


In [53]:
# dfinfo = dfmeta.merge(dfskip,on='period',suffixes=['_meta','']).groupby('period').sum()
# dfinfo
# # dfinfo_str=dfinfo.applymap(lambda x: f'{int(x):,}').rename_axis('Period (20yr)')
# # dfinfo_str.columns=['# Texts','# Sentences','# Words']
# # printm(dfinfo_str.to_markdown())

In [54]:
dfmeta.groupby('period').nunique()

,author,title,year,NumericPubDate,RecordID,Publication_PublicationID,Publication_Title,Publication_Qualifier,Publisher,VolumeIssue,...,LanguageCode,SourceType,num_words,path_freqs,path_txt,path_xml,_year_orig,corpus,Genre,count
period,,,,,,,,,,,,,,,,,,,,,
1680-1700,1,70,17,92,174,21,23,1,16,1,...,1,1,146,174,174,174,17,1,9,1
1700-1720,1,199,20,269,493,38,41,1,30,1,...,1,1,370,493,493,493,20,1,10,1
1720-1740,1,370,20,328,758,32,40,4,26,1,...,1,2,569,758,758,758,20,1,11,1
1740-1760,1,617,20,315,1143,36,46,5,31,1,...,1,2,765,1143,1143,1143,20,1,12,1
1760-1780,1,1017,20,282,1633,33,34,5,23,1,...,1,2,1019,1633,1633,1633,20,1,12,1
1780-1800,1,1309,20,392,2280,40,54,7,37,1,...,1,2,1299,2280,2280,2280,20,1,12,1
1800-1820,1,2256,20,465,3773,76,102,7,68,1,...,1,1,1891,3773,3773,3773,20,1,12,1
1820-1840,1,3097,20,768,5102,103,141,9,90,1,...,1,1,2423,5102,5102,5102,20,1,12,1
1840-1860,1,3400,20,697,5901,99,124,8,89,1,...,1,2,2735,5901,5901,5901,20,1,12,1


In [55]:
dfocr=pd.read_json('/home/ryan/lltk_data/corpora/bpo/data/data.ocr_accuracy.jsonl',lines=True).set_index('id')


In [60]:
def get_dfinfo_grp(by='period'):
    dfinfo = dfmeta.join(
        dfocr,rsuffix='_ocr'
    ).groupby(by).sum().join(
        dfskip.groupby('period').sum(),
        lsuffix='_meta'
    )[['num_words_ocr','num_recog','count','num_lines','num_words']].join(
        orig_stats
    )
#     dfinfo['num_journals']=dfmeta.groupby('period').Publication_PublicationID.nunique()
    dfinfo.loc['(Total)']=dfinfo.sum()
    dfinfo.loc['(Total)', 'num_journals']=dfmeta.Publication_PublicationID.nunique()

    dfinfo['ocr_accuracy'] = [x/y for x,y in zip(
        dfinfo.num_recog,dfinfo.num_words_ocr
    )]
    return dfinfo

In [61]:
dfinfo=get_dfinfo_grp('period')
dfinfo // 1000000

,num_words_ocr,num_recog,count,num_lines,num_words,num_journals,num_articles,ocr_accuracy
period,,,,,,,,
1680-1700,4.0,3.0,0.0,0.0,1.0,0.0,NaN,0.0
1700-1720,11.0,8.0,0.0,0.0,3.0,0.0,NaN,0.0
1720-1740,26.0,20.0,0.0,0.0,8.0,0.0,NaN,0.0
1740-1760,44.0,33.0,0.0,0.0,15.0,0.0,NaN,0.0
1760-1780,75.0,57.0,0.0,1.0,26.0,0.0,NaN,0.0
1780-1800,132.0,97.0,0.0,1.0,40.0,0.0,NaN,0.0
1800-1820,214.0,170.0,0.0,2.0,65.0,0.0,NaN,0.0
1820-1840,344.0,296.0,0.0,4.0,105.0,0.0,NaN,0.0
1840-1860,503.0,439.0,0.0,5.0,136.0,0.0,NaN,0.0


In [59]:
odf = dfinfo[[
    'num_journals','num_articles','num_lines','num_words',
    'num_words_ocr'
]].fillna(0).applymap(lambda x: f'{int(x):,}').rename_axis('Period')
odf['ocr_accuracy']=dfinfo.ocr_accuracy.apply(lambda x: f'{(x*100):.0f}%')
odf=odf[['num_journals','num_articles','num_words_ocr','ocr_accuracy']]#,'num_lines','num_words']]
odf.columns=['# Journals','# Articles','# Words','%OCR acc.']#, '# Sents (model)', '# Words (model)']
odf

,# Journals,# Articles,# Words,%OCR acc.
Period,,,,
1680-1700,0,0,"4,452,787",72%
1700-1720,0,0,"11,207,953",75%
1720-1740,0,0,"26,850,630",75%
1740-1760,0,0,"44,561,146",75%
1760-1780,0,0,"75,310,129",77%
1780-1800,0,0,"132,655,719",74%
1800-1820,0,0,"214,909,584",79%
1820-1840,0,0,"344,150,429",86%
1840-1860,0,0,"503,038,143",87%


In [38]:
# import a/dataframe_image as dfi

In [39]:
import dataframe_image as dfi
ofnfn=os.path.join(PATH_FIGS2,f'fig.corpus1-v9.{ybin}.png')
odf.dfi.export(ofnfn)
print_img(ofnfn)

NameError: name 'odf' is not defined

In [ ]:
meta=dfmeta.join(dfocr,rsuffix='_ocr')
# meta['Period']=meta.year.apply(lambda y: periodize_bystep(y,20,1700,1900))
# meta=meta[meta.Period!='']
meta['Period']=meta['period']
mdf=meta.groupby(['Period','Genre']).num_words_ocr.sum().reset_index()
mpiv=mdf.pivot('Genre','Period','num_words_ocr').fillna(0)
mpiv=mpiv[mpiv.columns[1:-1]]
for col in mpiv:
    mpiv[col]=mpiv[col]/mpiv[col].sum()
mpiv['sum']=mpiv.sum(axis=1)
mpiv=mpiv.sort_values('sum',ascending=False).drop('sum',1).rename_axis('Genre')
mpiv=mpiv.style.background_gradient('gray_r',vmin=0,vmax=1)
mpiv=mpiv.format('{:.2%}')
mpiv

In [ ]:
mpiv.export_png(os.path.join(PATH_FIGS2,'fig.corpus3-v9.png'))

In [ ]:
# dfmeta.style.background_gradient??

In [ ]:
dfmeta.num_words=pd.to_numeric(dfmeta.num_words)

In [ ]:
words=dfmeta.groupby('Genre').agg(['sum','mean']).drop('year',1)
words.columns=['_'.join(x) for x in words.columns.to_flat_index()]
words['perc']=words['num_words_sum'] / words['num_words_sum'].sum() * 100
words=words.sort_values('perc',ascending=False)
words=words.applymap(lambda x: f'{x:,.0f}')
words['perc']=words['perc'].apply(lambda x: f'{x}%')
words=words[['num_words_sum','perc','num_words_mean']]
words.columns=['# Words','% Words', '# Words (avg.)']
# print(f'\n\n{words.to_markdown()}\n\n')

In [ ]:
dfinfogenre = genres.join(words).sort_values('count',ascending=False).drop('count',1)
dfinfogenre

In [ ]:
dfi.export(dfinfogenre, os.path.join(PATH_FIGS2,'fig.corpus2.png'))

In [ ]:
print(dfinfogenre.to_markdown())

In [ ]:
dfmeta[dfmeta.ObjectType=='General Information'].index[0]

In [ ]:
t=C.textd['1642/General_Information/2032695']
t

In [ ]:
C.meta.num_words=C.meta.num_words.apply(float)
C.meta.num_words.mean()

In [ ]:
C.meta.groupby('ObjectType').sum()

In [ ]:
dfgpinfo=dfmeta.groupby(['period','Genre']).sum().reset_index()
dfgpinfo['year']=[int(y[:4]) for y in dfgpinfo.period]
dfgpinfo=pd.concat(grp.assign(perc_words=grp.num_words / grp.num_words.sum()) for i,grp in dfgpinfo.groupby('period'))
dfgpinfo

In [ ]:
dfgpinfo_mean=dfgpinfo.groupby('Genre').mean().sort_values('perc_words',ascending=False)
dfgpinfo_mean

In [ ]:
dfgpinfo['Genre']=pd.Categorical(dfgpinfo['Genre'], categories=dfgpinfo_mean.index)
dfgpinfo

In [ ]:
fig=p9.ggplot(
    p9.aes(x='period',y='perc_words',group='Genre'),
    data=dfgpinfo.query('1720<=year<1900')
)
fig+=p9.theme_minimal()
fig+=p9.theme(text=p9.element_text(size=6), axis_text_x=p9.element_text(rotation=90,size=6))
fig+=p9.geom_line()
fig+=p9.facet_wrap('Genre')
p9.options.figure_size=6,6
fig